# KG Quality Evaluation Runner
Imports functions from `/src` and evaluates two CSV-based KGs.

In [6]:
import os, sys
import pandas as pd

# Add repo root to sys.path (similar to your pattern)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Try to load PROJECT_ROOT; fallback to cwd
try:
    from app_settings import PROJECT_ROOT
except Exception:
    PROJECT_ROOT = os.getcwd()

os.chdir(PROJECT_ROOT)
print(f"Changed working dir to {PROJECT_ROOT}")


Changed working dir to C:\wamp\www\KGs_for_Vertical_AI


In [7]:
from src.kg_eval import evaluate_csv, format_report, interpret_kg_comparison_with_llm

## Configure CSVs & Columns

In [8]:

# CSV paths (based on your example)
RDB_CSV_PATH = "results/kgs/rdb_ontology_kg.csv"
TXT_CSV_PATH = "results/kgs/txt_ontology_kg.csv"

# Edge columns
SOURCE_COL = "source"
TARGET_COL = "target"

DIRECTED = False
HEAVY_METRICS = True


## Evaluate the two KGs

In [9]:

rdb_eval = evaluate_csv(
    RDB_CSV_PATH,
    source_col=SOURCE_COL,
    target_col=TARGET_COL,
    directed=DIRECTED,
    heavy_metrics=HEAVY_METRICS
)
print(format_report(rdb_eval))


File: results/kgs/rdb_ontology_kg.csv
Nodes: 406 | Edges: 203 | Density: 0.00246914
Isolated nodes: 0 | Connected components: 203
Largest component ratio: 0.005
Clustering: 0.000

-- Quality Ratio --
Overall: 0.701
  Connectivity: 1.000
  Density: 1.000
  Component: 0.005


In [10]:

txt_eval = evaluate_csv(
    TXT_CSV_PATH,
    source_col=SOURCE_COL,
    target_col=TARGET_COL,
    directed=DIRECTED,
    heavy_metrics=HEAVY_METRICS
)
print(format_report(txt_eval))


File: results/kgs/txt_ontology_kg.csv
Nodes: 438 | Edges: 219 | Density: 0.00228833
Isolated nodes: 0 | Connected components: 219
Largest component ratio: 0.005
Clustering: 0.000

-- Quality Ratio --
Overall: 0.701
  Connectivity: 1.000
  Density: 1.000
  Component: 0.005


## Side-by-side comparison

In [11]:

def to_flat_df(e):
    m = e['metrics'].copy()
    for k, v in e['quality_components'].items():
        m[f'QR_{k}'] = v
    m['QR_overall'] = e['quality_overall']
    m['path'] = e['path']
    return pd.DataFrame([m])

comp_df = pd.concat([to_flat_df(rdb_eval), to_flat_df(txt_eval)], ignore_index=True)
comp_df.set_index('path', inplace=True)
comp_df


,nodes,edges,density,isolated_nodes,connected_components,largest_component_ratio,avg_clustering,QR_ConnectivityScore,QR_DensityScore,QR_ComponentScore,QR_overall
path,,,,,,,,,,,
results/kgs/rdb_ontology_kg.csv,406,203,0.002469,0,203,0.004926,0.0,1.0,1.0,0.004926,0.701478
results/kgs/txt_ontology_kg.csv,438,219,0.002288,0,219,0.004566,0.0,1.0,1.0,0.004566,0.701370


### Knowledge Graph Metrics Explained

- **nodes**: Total number of unique entities (nodes) in the graph.
- **edges**: Total number of relationships (edges) connecting nodes in the graph.
- **density**: Ratio of actual edges to all possible edges. Higher density means more connections relative to graph size.
- **isolated_nodes**: Number of nodes with no connections (degree zero). High values may indicate disconnected or unused entities.
- **connected_components**: Number of separate subgraphs in the KG. A lower number is better for retrieval, as it means the graph is more unified.
- **largest_component_ratio**: Fraction of nodes in the largest connected component. Values close to 1 mean most nodes are reachable from each other.
- **avg_clustering**: Measures how likely nodes are to form tightly-knit groups. Higher values indicate more local connectivity and context.

**Quality Component Scores**  
- **QR_ConnectivityScore**: A normalized score (0–1) reflecting how well-connected the graph is, based on density and structure.  
- **QR_DensityScore**: A normalized score (0–1) for edge density, indicating how richly the graph is populated with relationships.  
- **QR_ComponentScore**: A normalized score (0–1) for the largest component ratio, showing how unified the graph is.  
- **QR_overall**: The overall quality score (0–1), calculated as a weighted average of the three component scores—connectivity, density, and largest component ratio. The combined score summarizes the structural health of the knowledge graph for retrieval tasks.


### Interpreting KG Quality Reports with Azure OpenAI

We use Azure OpenAI (GPT-4.1 Nano) to generate expert interpretations of the knowledge graph quality reports. The LLM analyzes the structural metrics and quality scores, then provides insights, highlights strengths and weaknesses, and suggests improvements for each graph.

This step helps translate raw metrics into actionable recommendations for improving knowledge graph design and retrieval performance.

In [12]:
# Get LLM comparative interpretation for both KG reports
rdb_report = format_report(rdb_eval)
txt_report = format_report(txt_eval)
comparison_revision = interpret_kg_comparison_with_llm(rdb_report, txt_report)
print("KG Comparative LLM Interpretation:\n", comparison_revision)

KG Comparative LLM Interpretation:
 Winner: Both KGs are effectively tied in overall quality score (0.701), with identical component scores, connectivity, density, and largest component ratio. However, the second report (txt_ontology_kg.csv) has slightly more nodes and edges, indicating a marginally larger and potentially more comprehensive graph.

Key Differences and Comparative Analysis:
- **Size and Connectivity**: The second KG has more nodes (438 vs. 406) and edges (219 vs. 203), suggesting it covers a broader set of entities and relationships.
- **Density**: The first KG has a slightly higher density (0.00246914 vs. 0.00228833), implying it is marginally more interconnected relative to its size.
- **Isolated Nodes**: Both have zero isolated nodes, indicating no disconnected entities.
- **Connected Components and Largest Component**: Both have the same number of components and the same largest component ratio (0.005), reflecting similar levels of graph unification.
- **Clustering 